# Assignment 1 – Backprop

In [49]:
#@title Library Imports [do not change]

import importlib 
!git clone https://www.github.com/rycolab/intro-nlp-f22.git
utils = importlib.import_module("intro-nlp-f22.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

fatal: destination path 'intro-nlp-f22' already exists and is not an empty directory.


In [50]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "0" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'x/y', 'in_vars': {'x': 1.0, 'y': 1.0}, 'output': 1.0, 'derivative': {'x': 1.0, 'y': -1.0}}
['x', '/', 'y'] {'x': 1.0, 'y': 1.0}


In [60]:
from ast import Not
#@title ToDo1: Building

class Builder():

    class Node():
        """
          If node is a constant then var=None
          if node is leaf then parents=None
        """
        def __init__(self, var, parents=[], operator: str=None, value: float=None):
            self.operator = operator
            self.var = var
            self.value = value

            self.parents = parents
            self.children = []
            for p in self.parents:
                p.children.append(self)

            # Holds the gradient, gradient[var]=partial wrt 'var'.
            self.grad = {}
        
        def __str__(self):
            if self.parents is None:
              self.parents = []
            return f'var: {str(self.var):6} op: {str(self.operator):6} val: {self.value} hash: {hash(self)} len: {len(self.children)}'


    def build_graph(self, infix, in_vars: dict, avail_vars: set()):
        """
          Builds the computational graph recursively.
          Values are not initiated here.
          Operators are strings, i.e not mapped to respective Operator class.
        """
        if not isinstance(infix, list):
            try:
                n = self.Node(None, value=float(infix))
            except ValueError:
                if not (infix in in_vars):
                    print(f'Variable \'{infix}\' is used but is not in \'in_vars\'.')
                # We must check that variable Node doesnt already exist!
                # e.g. if input is x+x
                # must be able to get a node from knowing its variable name
                # i.e mapping from variable name to Node object.
                if infix in self.in_nodes:
                    n = self.in_nodes[infix]
                else:
                    n = self.Node(infix)
                    self.in_nodes[infix] = n
            return n
          
        if len(infix) == 3:
            p1 = self.build_graph(infix[0], in_vars, avail_vars)
            p2 = self.build_graph(infix[2], in_vars, avail_vars)
            n = self.Node(avail_vars.pop(), [p1, p2],  operator=infix[1])
        elif len(infix) == 2:
            p = self.build_graph(infix[1], in_vars, avail_vars)
            n = self.Node(avail_vars.pop(), [p], operator=infix[0])
        else:
            raise NotImplementedError('Only unary and binary ops are supported')
        return n
    
    def topo_sort(self, node):
        """
            Return the computational graph in topological order.
        """
        sorted = []
        visited = set()

        def dfs_topo(node):
            """ Topo sort from node that can reach all others. """
            for p in node.parents:
                if p not in visited:
                    visited.add(p)
                    dfs_topo(p)
            sorted.append(node)         

        dfs_topo(node)
        return sorted

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)
        self.infix = infix
        self.in_nodes = {}
        print(self.avail_vars)

        g = self.build_graph(infix, in_vars, avail_vars)
        self.graph = self.topo_sort(g)


        print(self.in_nodes)
        def printg(g):
            print(g)
            if g.parents is None:
                return
            for p in g.parents:
                printg(p)
        printg(g)
        print('-'*50)
        for n in self.graph:
            print(n)
          
        
       # pass  ## ToDO: implement and set self.graph


if __name__ == '__main__':
    print(infix)
    g = Builder(infix, in_vars)
   # print(g.graph)
    #g = Builder([['x', '+', 'y'],'+',['z', '+', 'z2']], in_vars)
    
    

['x', '/', 'y']
['ψ', 'χ', 'φ', 'υ', 'τ', 'σ', 'ς', 'ρ', 'π', 'ο', 'ξ', 'ν', 'μ', 'λ', 'κ', 'ι', 'θ', 'η', 'ζ', 'ε', 'δ', 'γ', 'β', 'α', 'z', 'w', 'v', 'u', 't', 's', 'r', 'q', 'p', 'o', 'n', 'm', 'l', 'k', 'j', 'i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']
{'x': <__main__.Builder.Node object at 0x7f8e20819910>, 'y': <__main__.Builder.Node object at 0x7f8e20819b10>}
var: ν      op: /      val: None hash: 8765525465509 len: 0
var: x      op: None   val: None hash: 8765525465489 len: 1
var: y      op: None   val: None hash: 8765525465521 len: 1
--------------------------------------------------
var: x      op: None   val: None hash: 8765525465489 len: 1
var: y      op: None   val: None hash: 8765525465521 len: 1
var: ν      op: /      val: None hash: 8765525465509 len: 0


In [48]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        pass ## ToDO: implement

    def df(self, a, b = None):
        pass ## ToDO: implement

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b): # i did this
        return a / b

    def df(self, a, b): # i did this
        return [1.0 / b, -a / b**2]

class Add(Operator):

    def f(self, a, b): # i did this
        return a + b

    def df(self, a, b = None):
        pass ## ToDO: implement

class Sub(Operator):

    def f(self, a, b = None): # i did this
        if b is None:
            return -a # should be in multiply operator instead?
        return a-b

    def df(self, a, b = None):
        pass ## ToDO: implement

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None): # i did this
        return math.sin(a)

    def df(self, a, b=None):
        pass ## ToDO: implement

class Cos(Operator):

    def f(self, a, b=None):
        pass ## ToDO: implement

    def df(self, a, b=None):
        pass ## ToDO: implement

In [61]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self, ):
        for n in self.graph:
            if n.var in self.in_vars:
                n.value = self.in_vars[n.var]
            elif n.operator:
                inputs = [p.value for p in n.parents]
                assert len(inputs) == 1 or len(inputs) == 2
                n.value = self.fn_map[n.operator].f(*inputs)

        self.output = self.graph[-1].value

    ## backward execution____________________________

    def backward(self, ):

        for n in self.graph[::-1]:
            print(n)
            print(f'par: {len(n.parents)}, child: {len(n.children)}')
            if n.operator:
                inputs = [p.value for p in n.parents]
                assert len(inputs) == 1 or len(inputs) == 2
                # check if one of the inputs are constant?
                # that may affect derivative?
                grad = self.fn_map[n.operator].df(*inputs)

                # Can it happen that two inputs have gradient with len 1.
                # yes? if one of the inputs are constant?
                for p in n.parents:
                    if p in n.grad:
                        n.grad[p].append(grad.pop(0))
                    else:
                        n.grad[p] = [grad.pop(0)]
            elif n.var in self.in_vars: # Detta ska inte bara göras i leaf?
                grads = [c.grad for c in n.children]
                print(grads)
                #[{'node1': [2,3]}, {'node2':[1], 'node3':[4]}]
                n.grad = sum([sum(g[n]) for g in grads])

            if n.var in self.in_vars:
                self.derivative[n.var] = n.grad



if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

var: ν      op: /      val: 1.0 hash: 8765525465509 len: 0
par: 2, child: 0
var: y      op: None   val: 1.0 hash: 8765525465521 len: 1
par: 0, child: 1
[{<__main__.Builder.Node object at 0x7f8e20819910>: [1.0], <__main__.Builder.Node object at 0x7f8e20819b10>: [-1.0]}]
var: x      op: None   val: 1.0 hash: 8765525465489 len: 1
par: 0, child: 1
[{<__main__.Builder.Node object at 0x7f8e20819910>: [1.0], <__main__.Builder.Node object at 0x7f8e20819b10>: [-1.0]}]
1.0
{'y': -1.0, 'x': 1.0}


In [65]:
#@title Test Function for Debugging [do not change]

#utils.test_backprop(Builder, Executor, math_problem)
my_test = {'problem': 'x/x+x', 'in_vars': {'x': 5.0}, 'output': 6.0, 'derivative': {'x': 1.0}}
utils.test_backprop(Builder, Executor, my_test)

['ψ', 'χ', 'φ', 'υ', 'τ', 'σ', 'ς', 'ρ', 'π', 'ο', 'ξ', 'ν', 'μ', 'λ', 'κ', 'ι', 'θ', 'η', 'ζ', 'ε', 'δ', 'γ', 'β', 'α', 'z', 'y', 'w', 'v', 'u', 't', 's', 'r', 'q', 'p', 'o', 'n', 'm', 'l', 'k', 'j', 'i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']
{'x': <__main__.Builder.Node object at 0x7f8e207b5750>}
var: w      op: +      val: None hash: 8765525439965 len: 0
var: ν      op: /      val: None hash: 8765525439761 len: 1
var: x      op: None   val: None hash: 8765525439861 len: 3
var: x      op: None   val: None hash: 8765525439861 len: 3
var: x      op: None   val: None hash: 8765525439861 len: 3
--------------------------------------------------
var: x      op: None   val: None hash: 8765525439861 len: 3
var: ν      op: /      val: None hash: 8765525439761 len: 1
var: w      op: +      val: None hash: 8765525439965 len: 0
var: w      op: +      val: 6.0 hash: 8765525439965 len: 0
par: 2, child: 0


AttributeError: ignored

In [ ]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
FAILURE output: -1 != 1.0
FAILURE derivative: {} != {'x': 1.0, 'y': -1.0}

1: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
FAILURE output: -1 != 11.39
FAILURE derivative: {} != {'x': 7.39, 'y': -2.0}

2: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
FAILURE output: -1 != 32.0
FAILURE derivative: {} != {'x': 24.0, 'y': 8.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
FAILURE output: -1 != 0.14
FAILURE derivative: {} != {'x': -3.96, 'y': -0.99, 'z': 1.99}
